In [153]:
def cab_details_avai(ride):
    columns = ['Car type', 'Availability', 'Number of seats', 'ETA', 'Distance' ]
    ride_cat = ride['categories']
    dicti = []
    for i in range(0,len(ride_cat)):
        seats = "4"
        avail = "Yes"
        if ride_cat[i]['eta'] == -1:
            avail = "No"
            data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail}
            dicti.append(data)
        else:
            if ride_cat[i]['display_name']=="Share":
                seats = "1/2"
                data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail, 'Number of seats':           seats,'ETA' : ride_cat[i]['eta'],'Distance' : ride_cat[i]['distance']}
                dicti.append(data)   
            else:
                data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail, 'Number of seats': seats,'ETA' : ride_cat[i]['eta'],'Distance' : ride_cat[i]['distance']}
                dicti.append(data)
    df = pd.DataFrame(dicti,columns = columns)
    return df

In [154]:
def cab_details(ride):
    columns = ['Car type', 'Availability', 'Number of seats', 'Distance(Km)', 'Estimated Travel time(min)', 'Minimum Price', 'Maximum Price','ETA' ]
    #df = pd.DataFrame(data,columns = columns)
    ride_cat = ride['categories']
    ride_est = ride['ride_estimate']
    dict = []
    for i in range(0,len(ride_cat)):
        if ride_cat[i]['eta'] == 0:
            print (ride_cat[i]['display_name'] + " not available")
            data1 = {'Car type' : ride_cat[i]['display_name'], 'Availability': "No"}
            dict.append(data1)
        else:
            avail='Yes'
            if ride_cat[i]['eta'] == -1: avail='No'
            if ride_cat[i]['display_name'] == "Share":
                for j in range(0,2):
                    data1 = {'Car type': ride_cat[i]['display_name'],
                             'Number of seats': ride_est[i]['fares'][j]['seats'], 
                             'Travel time(min)': ride_est[i]['travel_time_min'], 
                             'Maximum Price': ride_est[i]['fares'][j]['cost'], 
                             'Minimum Price': ride_est[i]['fares'][j]['cost'], 
                             'Availability' : avail,
                             'ETA': ride_cat[i]['eta']}
                    dict.append(data1)
            else:
                data1 = {'Car type': ride_cat[i]['display_name'],
                         'Number of seats': "4",
                         'Distance(Km)': ride_est[i]['distance'], 
                         'Estimated Travel time(min)': ride_est[i]['travel_time_in_minutes'], 
                         'Maximum Price': ride_est[i]['amount_max'], 
                         'Minimum Price': ride_est[i]['amount_min'], 
                         'Availability' : avail,
                         'ETA': ride_cat[i]['eta']}
                dict.append(data1)          
    df = pd.DataFrame(dict,columns = columns)
    return df

In [155]:
def looksrc(source):
    api = Geocoding()
    source_loc = api.geocode(source)
    source_lat_long = source_loc[0]['geometry']['location']
    headers = {'X-APP-TOKEN' : "e957bc073c664db7b0a31f4b060235f5"}
    payload= {'pickup_lat': str(source_lat_long['lat']), 'pickup_lng':str(source_lat_long['lng'])}
    response = requests.get('http://sandbox-t.olacabs.com/v1/products', params=payload, headers=headers)
    cab_details_df=cab_details_avai(response.json())
    cab_details_df.fillna(0,axis=1,inplace=True)
    return cab_details_df

In [156]:
import requests
import gmaps
import pandas as pd
from gmaps import Geocoding
x=looksrc("Stack Route,Kormangala")

In [158]:
x[x['Availability']=='Yes']

,Car type,Availability,Number of seats,ETA,Distance
4,Share,Yes,1/2,29.0,9.5


In [ ]:
def look(source,destination):
    api = Geocoding()
    source_loc = api.geocode(source)
    destination_loc = api.geocode(destination)
    source_lat_long = source_loc[0]['geometry']['location']
    destination_lat_long = destination_loc[0]['geometry']['location']
    headers = {'X-APP-TOKEN' : "e957bc073c664db7b0a31f4b060235f5"}
    payload= {'pickup_lat': str(source_lat_long['lat']), 'pickup_lng':str(source_lat_long['lng']), 'drop_lat':str(destination_lat_long['lat']), 'drop_lng':str(destination_lat_long['lng'])}
    response = requests.get('http://sandbox-t.olacabs.com/v1/products', params=payload, headers=headers)
    cab_details_df=cab_details(response.json())
    cab_details_df.fillna(0,axis=1,inplace=True)
    return cab_details_df

In [123]:
import requests
import gmaps
import pandas as pd

In [124]:
from gmaps import Geocoding

In [125]:
x=look("Stack Route,Kormangala","Wipro, Electronic City")

In [117]:
x

,Car type,Availability,Number of seats,Distance(Km),Estimated Travel time(min),Minimum Price,Maximum Price,ETA
0,Micro,No,4,14.59,37.0,186,213,-1
1,Mini,No,4,14.59,37.0,290,333,-1
2,Sedan,No,4,14.59,37.0,271,311,-1
3,Prime Sedan,No,4,14.59,37.0,235,270,-1
4,Share,Yes,1,0.00,0.0,125,125,29
5,Share,Yes,2,0.00,0.0,138,138,29


In [129]:
x.dtypes

Car type                       object
Availability                   object
Number of seats                object
Distance(Km)                  float64
Estimated Travel time(min)    float64
Minimum Price                   int64
Maximum Price                   int64
ETA                             int64
dtype: object

In [120]:
dftostring(x)

' Micro No 4 14.59 37.0 186 213 -1 \n Mini No 4 14.59 37.0 290 333 -1 \n Sedan No 4 14.59 37.0 271 311 -1 \n Prime Sedan No 4 14.59 37.0 235 270 -1 \n Share Yes 1 0.0 0.0 125 125 29 \n Share Yes 2 0.0 0.0 138 138 29 \n'

In [119]:
def dftostring(cab_details_df):
    cab_det=""
    for i in range(cab_details_df.shape[0]):
        x=list(cab_details_df.loc[i])
        s=" "
        for i in x:
            s=s+str(i)+" "
        cab_det=cab_det+s+"\n" 
    return cab_det 

In [139]:
import random

In [140]:
from random import randint

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [141]:
def book(source,destination,cabtype,noofseats):
    df=look(source,destination)
    driver_name=["Tharoon Veerasethu","Sharmaji","Chachaji","Chunnu","Munni","beta","daadaji","daadi","bhai","shinde","langoor"]
    for i in range(df.shape[0]):
        x=list(df.loc[i])
        if cabtype in x:
            for j in x:
                if j == cabtype and df.loc[i,'Availability']=='Yes':
                    SessInfo['bookingid']="OLA"+str(random_with_N_digits(6))
                    SessInfo['drivername']=random.choice(driver_name)
                    SessInfo['driverno']=random_with_N_digits(10)
                    SessInfo['carno']="KN-"+str(random_with_N_digits(2))+"-"+str(random_with_N_digits(4))
                    SessInfo['fare']=(df.loc[i,'Minimum Price']+df.loc[i,'Maximum Price'])/2
                    SessInfo['cabtype']=cabtype
                    SessInfo['noofseats']=noofseats
                    SessInfo['eta']=df.loc[i,'ETA']
                    if cabtype != 'Share': 
                        SessInfo['distance']=df.loc[i,'Distance(Km)']
                        Sessinfo['ett']=df.loc[i,'Estimated Travel time(min)']
                    break                    

In [142]:
SessInfo =dict()

In [143]:
book("Stack Route,Kormangala","Wipro, Electronic City","Share",1)

In [144]:
SessInfo

{'bookingid': 'OLA492865',
 'cabtype': 'Share',
 'carno': 'KN-98-1355',
 'drivername': 'shinde',
 'driverno': 8373442571,
 'eta': 29,
 'fare': 198.0,
 'noofseats': 1}

In [ ]:
import pandas as pd
df=

In [87]:
import requests

headers = {'X-APP-TOKEN' : "38056236716e4019861471e180aa8a0a"}
payload= {'pickup_lat': 12.9279, 'pickup_lng': 77.6271,}
#response = requests.get('http://sandbox-t.olacabs.com/v1/products', params=payload, headers=headers)
#response = requests.get('http://sandbox-t.olacabs.com/v1/products', params=payload, headers=headers)
res #= response.json()
res

{'categories': [{'currency': 'INR',
   'display_name': 'Micro',
   'distance': -1,
   'distance_unit': 'kilometre',
   'eta': -1,
   'fare_breakup': [{'base_fare': 50.0,
     'cost_per_distance': 9.0,
     'minimum_distance': 0.0,
     'minimum_fare': 60.0,
     'minimum_time': 2.0,
     'rates_higher_than_usual': False,
     'ride_cost_per_minute': 2.0,
     'surcharge': [],
     'type': 'flat_rate',
     'waiting_cost_per_minute': 0.0}],
   'id': 'micro',
   'image': 'http://d1foexe15giopy.cloudfront.net/micro.png',
   'ride_later_enabled': 'false',
   'time_unit': 'minute'},
  {'currency': 'INR',
   'display_name': 'Mini',
   'distance': -1,
   'distance_unit': 'kilometre',
   'eta': -1,
   'fare_breakup': [{'base_fare': 50.0,
     'cost_per_distance': 5.0,
     'minimum_distance': 4.0,
     'minimum_fare': 60.0,
     'minimum_time': 2.0,
     'rates_higher_than_usual': False,
     'ride_cost_per_minute': 2.0,
     'surcharge': [],
     'type': 'flat_rate',
     'waiting_cost_per_mi

In [68]:
payload= {'pickup_lat': 12.9279, 'pickup_lng': 77.6271, 'drop_lat':12.96, 'drop_lng':77.678}
response = requests.get('http://sandbox-t.olacabs.com/v1/products', params=payload, headers=headers)

In [69]:
m = response.json()

In [72]:
m['categories'][0]

{'currency': 'INR',
 'display_name': 'Micro',
 'distance': -1,
 'distance_unit': 'kilometre',
 'eta': -1,
 'fare_breakup': [{'base_fare': 50.0,
   'cost_per_distance': 9.0,
   'minimum_distance': 0.0,
   'minimum_fare': 60.0,
   'minimum_time': 2.0,
   'rates_higher_than_usual': False,
   'ride_cost_per_minute': 2.0,
   'surcharge': [],
   'type': 'flat_rate',
   'waiting_cost_per_minute': 0.0}],
 'id': 'micro',
 'image': 'http://d1foexe15giopy.cloudfront.net/micro.png',
 'ride_later_enabled': 'false',
 'time_unit': 'minute'}

In [40]:
m['ride_estimate']

[{'amount_max': 171,
  'amount_min': 149,
  'category': 'mini',
  'distance': 9.74,
  'travel_time_in_minutes': 32},
 {'amount_max': 250,
  'amount_min': 218,
  'category': 'sedan',
  'distance': 9.74,
  'travel_time_in_minutes': 32},
 {'amount_max': 244,
  'amount_min': 213,
  'category': 'micro',
  'distance': 9.74,
  'travel_time_in_minutes': 32},
 {'amount_max': 203,
  'amount_min': 177,
  'category': 'prime',
  'distance': 9.74,
  'travel_time_in_minutes': 32},
 {'category': 'share',
  'fares': [{'actual_cost': 154,
    'cost': 123,
    'discount_string': '21% of micro rate',
    'fare_string': '79% of micro rate',
    'peak_time_charge': '1.0x',
    'peak_time_text': 'Fare inclusive of 1.0x peak pricing',
    'savings': 31,
    'seats': 1},
   {'actual_cost': 154,
    'cost': 136,
    'discount_string': '12% of micro rate',
    'fare_string': '88% of micro rate',
    'peak_time_charge': '1.0x',
    'peak_time_text': 'Fare inclusive of 1.0x peak pricing',
    'savings': 18,
    's

In [ ]:
def ola_available(source, destination=None):
    